In [1]:
%matplotlib inline

import cv2
import keras
from keras.applications.imagenet_utils import preprocess_input
from keras.backend.tensorflow_backend import set_session
from keras.models import Model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import pickle
import glob
import os
from random import shuffle
from scipy.misc import imread
from scipy.misc import imresize
from sklearn.utils import shuffle
import pandas as pd
import tensorflow as tf

from ssd import SSD300
from ssd_training import MultiboxLoss
from ssd_utils import BBoxUtility


plt.rcParams['figure.figsize'] = (8, 8)
plt.rcParams['image.interpolation'] = 'nearest'

%load_ext autoreload
%autoreload 2

np.set_printoptions(suppress=True)

# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.9
# set_session(tf.Session(config=config))

Using TensorFlow backend.
/a/h/jsendi01/Envs/deep-venv/lib/python2.7/site-packages/sklearn/externals/joblib/_multiprocessing_helpers.py:28: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [2]:
# some constants
INPUT_WIDTH = 300
INPUT_HEIGHT = 300 
NUM_CLASSES = 8 + 1 # 1 because of background (needed for training)
DATA_PATH = '/a/data/fisheries_monitoring/data/'

input_shape = (INPUT_HEIGHT, INPUT_WIDTH, 3)
classes = ['ALB', 'DOL', 'NoF', 'SHARK', 'BET', 'LAG', 'OTHER', 'YFT']

In [3]:
priors = pickle.load(open('prior_boxes_ssd300.pkl', 'rb'))
bbox_util = BBoxUtility(NUM_CLASSES, priors)

In [4]:
aug_folders = glob.glob(DATA_PATH + 'localizers/*')
for i, folder in enumerate(aug_folders):
    print "index:", i, '\t', "folder name:", folder
    
aug_folders = aug_folders[1:]

index: 0 	folder name: /a/data/fisheries_monitoring/data/localizers/splice
index: 1 	folder name: /a/data/fisheries_monitoring/data/localizers/invert
index: 2 	folder name: /a/data/fisheries_monitoring/data/localizers/vflip
index: 3 	folder name: /a/data/fisheries_monitoring/data/localizers/add
index: 4 	folder name: /a/data/fisheries_monitoring/data/localizers/emboss
index: 5 	folder name: /a/data/fisheries_monitoring/data/localizers/gaussianNoise
index: 6 	folder name: /a/data/fisheries_monitoring/data/localizers/blur
index: 7 	folder name: /a/data/fisheries_monitoring/data/localizers/original
index: 8 	folder name: /a/data/fisheries_monitoring/data/localizers/dropout
index: 9 	folder name: /a/data/fisheries_monitoring/data/localizers/rotate


In [52]:
def data_generator(batch_size, bbox_util, boxes, classes, INPUT_WIDTH, INPUT_HEIGHT):
    while True:
        #img_batch = np.zeros((batch_size, INPUT_HEIGHT, INPUT_WIDTH, 3))
        #box_batch = np.zeros((batch_size, 4))
        img_batch = []
        target_batch = []
        for file_name, value in boxes.iteritems():
            # Obtain label for that file (remember file path in pattern aug_type/label/frame.png)
            label = file_name.split('/')[1]
            # Then create a one-hot encoding vector for the classes
            label_vector = [1 if i == label else 0 for i in classes]

            # Read img and preprocess it
            path = DATA_PATH + 'localizers/' + file_name
            img = image.load_img(path)
                
            width, height = img.size
            img = img.resize((INPUT_HEIGHT, INPUT_WIDTH))
            img = image.img_to_array(img)
            img /= 255
            img_batch.append(img)
            
            # Read boxes (could be multiple of them) and normalize its measures
            #rows = boxes.loc[boxes['img'] == file_name]
            #del rows['img']
            #rows.loc[:, 'x'] = rows['x'] / width
            #rows.loc[:, 'y'] = rows['x'] / height
            #rows.loc[:, 'w'] = rows['x'] / width
            #rows.loc[:, 'h'] = rows['x'] / height
            
            #rows = np.array(rows)
            #rows[:, 2] = rows[:, 2] + rows[:, 0] # x2 = x1 + w
            #rows[:, 3] = rows[:, 3] + rows[:, 1] # y2 = y1 + h
            #target = np.concatenate((rows, 
            #                         np.tile(label_vector[np.newaxis,:], (rows.shape[0],1))), 
            #                        axis=1)

            #target = bbox_util.assign_boxes(target)
            old_x, old_y, old_w, old_h = value
            new_x = old_x / width
            new_y = old_y / height
            new_w = old_w / width
            new_h = old_h / height
            
            target = [new_x, new_y, new_x + new_w, new_y + new_h] + label_vector
            target = bbox_util.assign_boxes(np.array(target)[np.newaxis, :])
            
            target_batch.append(target)
            
            if len(target_batch) == batch_size:
                tmp_img_batch = np.array(img_batch)
                tmp_target_batch = np.array(target_batch)
                img_batch = []
                target_batch = []
                yield (tmp_img_batch, tmp_target_batch)
                
        
def load_all_labels(aug_folders):
    all_targets = pd.DataFrame(columns = ["img", "x","y","w","h"])
    for folder in aug_folders:
        folder_name = os.path.basename(folder)
        
        print "Loading data augmentation folder:", folder_name
        targets = pd.read_csv(folder + '/boxes.csv', names = ["img", "x","y","w","h"])
        targets = targets.sort_values(by = "img")
        targets["img"] = folder_name + '/' + targets["img"]
        print "Number of examples:", len(targets)
        print
        
        all_targets = all_targets.append(targets)
    print "total number of examples: ", len(all_targets)
    return all_targets

def train_val_test_split(all_labels, val_size, test_size):
    all_labels = shuffle(all_labels)
    test_labels = all_labels[0:test_size]
    val_labels = all_labels[test_size:test_size + val_size]
    train_labels = all_labels[test_size + val_size:]
    return train_labels, val_labels, test_labels


In [38]:
#gt = pickle.load(open('gt_pascal.pkl', 'rb'))
#keys = sorted(gt.keys())
#num_train = int(round(0.8 * len(keys)))
#train_keys = keys[:num_train]
#val_keys = keys[num_train:]
#num_val = len(val_keys)

all_labels = load_all_labels(aug_folders)
train_labels, val_labels, test_labels = train_val_test_split(all_labels, 2000, 1000)

train_labels = train_labels.set_index('img').T.to_dict('list')
val_labels = val_labels.set_index('img').T.to_dict('list')
test_labels = test_labels.set_index('img').T.to_dict('list')

Loading data augmentation folder: invert
Number of examples: 8742

Loading data augmentation folder: vflip
Number of examples: 17484

Loading data augmentation folder: add
Number of examples: 8742

Loading data augmentation folder: emboss
Number of examples: 4371

Loading data augmentation folder: gaussianNoise
Number of examples: 13113

Loading data augmentation folder: blur
Number of examples: 8742

Loading data augmentation folder: original
Number of examples: 4371

Loading data augmentation folder: dropout
Number of examples: 8742

Loading data augmentation folder: rotate
Number of examples: 30597

total number of examples:  104904


In [45]:
batch_size = 30
train_steps = np.ceil(len(train_labels)/batch_size)

gen = data_generator(batch_size, bbox_util, train_labels, classes, INPUT_HEIGHT, INPUT_WIDTH)

In [48]:
model = SSD300(input_shape, num_classes=NUM_CLASSES)
model.load_weights(DATA_PATH + 'models/localizers/weights_SSD300.hdf5', by_name=True)

In [49]:
freeze = ['input_1', 'conv1_1', 'conv1_2', 'pool1',
          'conv2_1', 'conv2_2', 'pool2',
          'conv3_1', 'conv3_2', 'conv3_3', 'pool3']#,
#           'conv4_1', 'conv4_2', 'conv4_3', 'pool4']

for L in model.layers:
    if L.name in freeze:
        L.trainable = False

In [18]:
def schedule(epoch, decay=0.9):
    return base_lr * decay**(epoch)

callbacks = [keras.callbacks.ModelCheckpoint('./checkpoints/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
                                             verbose=1,
                                             save_weights_only=True),
             keras.callbacks.LearningRateScheduler(schedule)]

In [19]:
base_lr = 3e-4
optim = keras.optimizers.Adam(lr=base_lr)
# optim = keras.optimizers.RMSprop(lr=base_lr)
# optim = keras.optimizers.SGD(lr=base_lr, momentum=0.9, decay=decay, nesterov=True)
model.compile(optimizer=optim,
              loss=MultiboxLoss(NUM_CLASSES, neg_pos_ratio=2.0).compute_loss)

In [20]:
nb_epoch = 30
val_gen = data_generator(batch_size, bbox_util, val_labels, classes, INPUT_WIDTH, INPUT_HEIGHT)
val_steps = np.ceil(len(val_labels)/batch_size)

history = model.fit_generator(gen, len(train_labels),
                              nb_epoch, verbose=2,
                              callbacks=callbacks,
                              validation_data=val_gen,
                              nb_val_samples=len(val_labels),
                              nb_worker=1)

Epoch 1/30


/a/h/jsendi01/Envs/deep-venv/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


KeyboardInterrupt: 

In [ ]:
inputs = []
images = []
img_path = path_prefix + sorted(val_keys)[0]
img = image.load_img(img_path, target_size=(300, 300))
img = image.img_to_array(img)
images.append(imread(img_path))
inputs.append(img.copy())
inputs = preprocess_input(np.array(inputs))

In [ ]:
preds = model.predict(inputs, batch_size=1, verbose=1)
results = bbox_util.detection_out(preds)

In [ ]:
for i, img in enumerate(images):
    # Parse the outputs.
    det_label = results[i][:, 0]
    det_conf = results[i][:, 1]
    det_xmin = results[i][:, 2]
    det_ymin = results[i][:, 3]
    det_xmax = results[i][:, 4]
    det_ymax = results[i][:, 5]

    # Get detections with confidence higher than 0.6.
    top_indices = [i for i, conf in enumerate(det_conf) if conf >= 0.6]

    top_conf = det_conf[top_indices]
    top_label_indices = det_label[top_indices].tolist()
    top_xmin = det_xmin[top_indices]
    top_ymin = det_ymin[top_indices]
    top_xmax = det_xmax[top_indices]
    top_ymax = det_ymax[top_indices]

    colors = plt.cm.hsv(np.linspace(0, 1, 4)).tolist()

    plt.imshow(img / 255.)
    currentAxis = plt.gca()

    for i in range(top_conf.shape[0]):
        xmin = int(round(top_xmin[i] * img.shape[1]))
        ymin = int(round(top_ymin[i] * img.shape[0]))
        xmax = int(round(top_xmax[i] * img.shape[1]))
        ymax = int(round(top_ymax[i] * img.shape[0]))
        score = top_conf[i]
        label = int(top_label_indices[i])
#         label_name = voc_classes[label - 1]
        display_txt = '{:0.2f}, {}'.format(score, label)
        coords = (xmin, ymin), xmax-xmin+1, ymax-ymin+1
        color = colors[label]
        currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, linewidth=2))
        currentAxis.text(xmin, ymin, display_txt, bbox={'facecolor':color, 'alpha':0.5})
    
    plt.show()